In [ ]:
# Set up API environment
import requests
from bedrock.client.client import BedrockAPI
SERVER = "http://localhost:81/"
api = BedrockAPI(SERVER)

In [ ]:
# Upload source file to Bedrock server
filepath = 'Rand2011PNAS_cooperation_data.csv'
ingest_id = 'opals.spreadsheet.Spreadsheet.Spreadsheet'
resp = api.put_source('Rand2011', ingest_id, 'default', {'file': open(filepath, "rb")})
if resp.status_code == 201:
    print('Source {0} successfully uploaded'.format(filepath))
else:
    print('Failed to upload {0}'.format(filepath)) 

In [ ]:
available_sources = api.list("dataloader", "sources").json()
source_id = available_sources[0]['src_id']
endpoint = api.endpoint("dataloader", "sources/%s/explore/" % source_id)
resp = requests.get(endpoint)
print("Sources")
print([i for i in resp.json().keys()])

In [ ]:
# Create new matrix using the source
source_name = 'Rand2011PNAS_cooperation_data'
matrix_id = 'rand_mtx'
matrix_name = 'rand_mtx'
feature_list = 'sessionnum,condition,playerid,decision0d1c,previous_decision,round_num,num_neighbors,group_size,fluid_dummy,_Icondition_2,_Icondition_3,_Icondition_4'.split(',')
column_types = ['Numeric' for x in feature_list]
matrix_filters = dict.fromkeys(feature_list,{})
matbody = {
    'matrixFeatures': feature_list,
    'matrixFeaturesOriginal': feature_list,
    'matrixFilters': matrix_filters,
    'matrixName': matrix_name,
    'matrixTypes': column_types,
    'sourceName': source_name
}
url = api.endpoint("dataloader", "sources/%s" % (source_id))
resp = requests.post(url, json=matbody)
if resp.status_code == 201:
    print("Matrix successfully created")
    source = resp.json()[0]
else:
    print("Error creating matrix")

In [ ]:
# Apply specified analysis to the matrix
analytic_id = "opals.logit2.Logit2.Logit2"
analysis_postdata = {
    'inputs': {
        'matrix.csv': source,
        'features.txt': source
    },
    'name': 'rand-logit2',
    'parameters': [{"attrname":"step","value":"2"}],
    'src': [source]
}
resp = api.post("analytics", "analytics/%s" % analytic_id, json=analysis_postdata)
result = resp.json()
if resp.status_code == 201:
    print("Logit2 analysis successful")
else:
    print("Logit2 analysis failed")
    print(result)

In [ ]:
# Download result matrix
url = api.endpoint("analytics", "results/download/%s/%s/%s" % (result['id'],'matrix.csv','logit2_result.csv'))
resp = requests.get(url)
print(url)

# Display result
import pandas
print(pandas.DataFrame([x.split(',') for x in resp.text.split("\n")]))